# 🧠 Notebook Technique - Implémentation du RAG Agentique

Bienvenue dans ce notebook, compagnon technique du mémoire intitulé **"Développement d’un Assistant Intelligent interne à BNP Paribas El Djazair"**.  
Ce document contient l'ensemble du code opérationnel lié à l’implémentation du système **RAG (Retrieval-Augmented Generation)** avec une **architecture multi-agents** telle que décrite dans le mémoire.

---

## 📌 À propos de ce notebook

- Ce notebook sert de **complément technique** au mémoire, permettant de **visualiser et tester** les différentes composantes du système RAG agentique.
- Chaque agent présenté dans le mémoire (ex. : Agent de recherche, Agent de synthèse, etc.) est implémenté ici de manière fonctionnelle.
- Le pipeline complet du système RAG y est codé, depuis l’extraction des données jusqu’à la génération augmentée.

## 🧱 Structure du code

Le notebook est structuré comme suit :
1. **Importations & configuration**
2. **Chargement et vectorisation des données**
3. **Définition des agents**
4. **Mécanisme d'orchestration' et exécution**
5. **Tests et démonstrations**

---

## ⚠️ Remarques

- Tous les blocs de code sont **fonctionnels** et peuvent être exécutés dans l'ordre.
- Certaines cellules peuvent nécessiter un accès à des **fichiers locaux**.
- Des **commentaires explicites** sont ajoutés dans chaque section pour faciliter la navigation dans le code.

---

> 📚 Pour les détails théoriques et le choix des architectures, veuillez vous référer au chapitre [5] du mémoire.


### Import des bibliothèques utilisés

In [ ]:
import os, pickle, fitz
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import T5ForConditionalGeneration
import PyPDF2
from typing import List, Optional, Tuple
import re
from typing import ClassVar, Any
import string
from string import punctuation
import spacy
from langchain.llms.base import LLM
from litellm import completion
from crewai import Agent, Task, Crew
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from crewai import Agent

## Préparation, pretraitement et stockage des données 

In [ ]:
def extract_text_from_pdf_folders(parent_folder: str) -> List[Tuple[str, str, int, str]]:

    documents = []
    for domaine in os.listdir(parent_folder):
        domaine_path = os.path.join(parent_folder, domaine)
        if os.path.isdir(domaine_path):
            for filename in os.listdir(domaine_path):
                if filename.lower().endswith(".pdf"):
                    pdf_path = os.path.join(domaine_path, filename)
                    doc = fitz.open(pdf_path)
                    for page_number, page in enumerate(doc, start=1):
                        page_text = page.get_text("text")
                        if page_text and page_text.strip():
                            documents.append((
                                domaine,              
                                filename,             
                                page_number,          
                                page_text.strip()    
                            ))
    return documents

### Cleaning des textes extraits 

In [ ]:
nlp = spacy.load("fr_core_news_sm")

# Stopwords spécifiques à conserver en mode "lite"
stop_keep = {'mais', 'donc', 'cependant', 'si', 'car'}

def clean_text(text, mode="lite"):
    # Normalisation de base
    text = re.sub(r"\s+", " ", text.strip())
    doc = nlp(text)

    if mode == "hard":
        # Supprimer tous les stopwords et mots courts, conserver uniquement les mots informatifs
        tokens = [
            token.text.lower()
            for token in doc
            if not token.is_stop and not token.is_punct and not token.is_space and len(token.text) > 2
        ]
    elif mode == "lite":
        # Supprimer tous les stopwords sauf les connecteurs utiles
        tokens = [
            token.text.lower()
            for token in doc
            if not token.is_punct and not token.is_space and (not token.is_stop or token.text.lower() in stop_keep)
        ]

    return " ".join(tokens)

### Chunking sémentique intelligent

In [ ]:
def hybrid_chunking(text, min_chunk_size=500, max_chunk_size=1000):
    """Segmentation par paragraphes, puis fallback phrase à phrase."""
    paragraphs = [p.strip() for p in text.split('\n') if len(p.strip()) > 0]
    chunks, current_chunk = [], ""

    for para in paragraphs:
        if len(para) > max_chunk_size:
            # découper en phrases
            for sent in nlp(para).sents:
                sentence = sent.text.strip()
                if len(current_chunk) + len(sentence) <= max_chunk_size:
                    current_chunk += " " + sentence
                else:
                    if len(current_chunk.strip()) >= min_chunk_size:
                        chunks.append(current_chunk.strip())
                        current_chunk = sentence
                    else:
                        current_chunk += " " + sentence
            if current_chunk.strip():
                chunks.append(current_chunk.strip())
                current_chunk = ""
        else:
            if len(current_chunk) + len(para) <= max_chunk_size:
                current_chunk += " " + para
            else:
                if len(current_chunk.strip()) >= min_chunk_size:
                    chunks.append(current_chunk.strip())
                    current_chunk = para
                else:
                    current_chunk += " " + para

    if current_chunk.strip():
        chunks.append(current_chunk.strip())

    return chunks

In [ ]:
def is_informative(text, min_keywords=2, min_length=30):
    doc = nlp(text)
    # Supprime la dépendance aux entités
    keywords = [token for token in doc if len(token.text) > 4]
    # Accepte aussi les chunks longs même sans keywords
    return len(keywords) >= min_keywords or len(text.split()) >= min_length

In [ ]:
def fuse_chunks(chunks, max_len=900):
    fused = []
    temp_text = ""
    current_file = None
    pages = set()

    for chunk in chunks:
        file = chunk["file"]
        page = chunk["page"]
        content = chunk["content_raw"]

        # Vérifie si on peut fusionner
        same_file = (file == current_file or current_file is None)
        temp_len = len(temp_text.split()) + len(content.split())
        can_fuse = (temp_len < max_len) and same_file

        if can_fuse:
            temp_text += " " + content
            pages.add(page)
        else:
            if temp_text:
                page_range = f"{min(pages)}" if len(pages) == 1 else f"{min(pages)}-{max(pages)}"
                fused.append({
                    "file": current_file,
                    "page": page_range,
                    "content": temp_text.strip()
                })
        # Réinitialisation
        temp_text = content
        current_file = file
        pages = {page}

    # Ajouter le dernier
    if temp_text:
        page_range = f"{min(pages)}" if len(pages) == 1 else f"{min(pages)}-{max(pages)}"
        fused.append({
            "file": current_file,
            "page": page_range,
            "content": temp_text.strip()
        })

    return fused

In [ ]:
parent_folder = r'lien vers le fichier qui contient vos documents'
embedding_dim = 1024
model = SentenceTransformer("BAAI/bge-m3", trust_remote_code=True)

In [ ]:
def detect_domains(base_folder):
    return [
        name for name in os.listdir(base_folder)
        if os.path.isdir(os.path.join(base_folder, name))
    ]

domains = detect_domains(parent_folder)

In [ ]:
indices = {}
embeddings = {}
chunks = {}

for domain in domains:
    prefix = f"{domain}_"
    idx_file = prefix + "faiss_index.index"
    emb_file = prefix + "embeddings.npy"
    meta_file = prefix + "metadata.pkl"

    print(f"Création d'un nouveau index pour {domain}.")
    indices[domain] = faiss.IndexFlatIP(embedding_dim)
    embeddings[domain] = np.array([]).reshape(0, embedding_dim)
    chunks[domain] = []

In [ ]:
docs = extract_text_from_pdf_folders(parent_folder)

# Trie par domaine
from collections import defaultdict
chunks_by_domain = defaultdict(list)
for domain, filename, page, raw_text in docs:
    raw_chunks = hybrid_chunking(raw_text)
    fused_chunks = fuse_chunks([{'file': filename, 'page': page, 'content_raw': c} for c in raw_chunks])
    for i, chunk in enumerate(fused_chunks):
        cleaned_hard = clean_text(chunk['content'], mode="hard")
        cleaned_lite = clean_text(chunk['content'], mode="lite")
        if len(cleaned_hard) < 10:
            continue  # évite les chunks inutiles
        chunks_by_domain[domain].append({
            "domaine": domain,
            "file": chunk['file'],
            "page": chunk['page'],
            "chunk_id": i,
            "content_raw": chunk['content'].strip(),
            "content_lite": cleaned_lite,
            "content_clean": cleaned_hard,
            "length": len(cleaned_hard),
            "nb_words": len(cleaned_hard.split()),
        })

#### Bloc de Stockage des Données Vectorisées

In [ ]:
# === Génération des embeddings normalisés ===
for domain, domain_chunks in chunks_by_domain.items():
    if not domain_chunks:
        print(f"❌ Aucun chunk pour {domain}")
        continue

    texts = [chunk['content_clean'] for chunk in domain_chunks]
    embeddings = model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)

    # Création FAISS index
    index = faiss.IndexFlatIP(embedding_dim)
    index.add(embeddings)

    # Sauvegarde
    prefix = f"{domain}_"
    faiss.write_index(index, prefix + "faiss_index.index")
    np.save(prefix + "embeddings.npy", embeddings)
    with open(prefix + "metadata.pkl", "wb") as f:
        pickle.dump(domain_chunks, f)

    print(f"✅ {len(domain_chunks)} chunks indexés pour le domaine {domain}")

### chargement des données direct si les les vecteurs existent deja 

In [ ]:
indices = {}
embeddings = {}
chunks = {}

for domain in domains:
    prefix = f"{domain}_"
    idx_file = prefix + "faiss_index.index"
    emb_file = prefix + "embeddings.npy"
    meta_file = prefix + "metadata.pkl"

    print(f"✅ Index FAISS existant chargé pour {domain}.")
    indices[domain] = faiss.read_index(idx_file)
    embeddings[domain] = np.load(emb_file)
    with open(meta_file, "rb") as f:
        chunks[domain] = pickle.load(f)

In [ ]:
# Load BGE reranker model
reranker_tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-reranker-large")
reranker_model = AutoModelForSequenceClassification.from_pretrained("BAAI/bge-reranker-large")

ici vous trouverez les fonctions retrieve du RAG utilisé dans l'agent de recherche

In [ ]:
class RetrieverAgent:
    def __init__(self, model, index, nlp, indexed_chunks, reranker_model, reranker_tokenizer, fallback_k=10):
        self.model = model
        self.index = index
        self.nlp = nlp
        self.indexed_chunks = indexed_chunks
        self.reranker_model = reranker_model
        self.reranker_tokenizer = reranker_tokenizer
        self.fallback_k = fallback_k

    def retrieve(self, query, k=3, keywords_boost=True, similarity_threshold=0.35):
        q_embed = self.model.encode(query, convert_to_numpy=True)
        D, I = self.index.search(np.array([q_embed]), k)

        results = [
            (self.indexed_chunks[i], float(D[0][j]))
            for j, i in enumerate(I[0])
            if float(D[0][j]) > similarity_threshold
        ]

        # Dédupliquer
        seen = {}
        for chunk, score in results:
            key = chunk["content_clean"]
            if key not in seen or score > seen[key][1]:
                seen[key] = (chunk, score)

        retrieved = [v[0] for v in sorted(seen.values(), key=lambda x: -x[1])]

        # Boost par mots-clés
        if keywords_boost and retrieved:
            doc = self.nlp(query)
            keywords = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct and len(token.text) > 2]

            boosted, normal = [], []
            for chunk in retrieved:
                content_lower = chunk["content_clean"].lower()
                if any(keyword in content_lower for keyword in keywords):
                    boosted.append(chunk)
                else:
                    normal.append(chunk)

            retrieved = boosted + normal

        # Fallback brut si rien trouvé
        if not retrieved:
            raw_hits = []
            query_lower = query.lower()
            for chunk in self.indexed_chunks:
                if all(term in chunk["content_lite"].lower() for term in query_lower.split() if len(term) > 2):
                    raw_hits.append(chunk)
            if raw_hits:
                return raw_hits
            # Dernier recours
            partial_hits = [
                chunk for chunk in self.indexed_chunks
                if any(term in chunk["content_lite"].lower() for term in query_lower.split() if len(term) > 2)
            ]
            return partial_hits[:k] if partial_hits else []

        return retrieved

    def rerank(self, query, passages):
        if not passages:
            return []

        texts = [p["content_lite"] for p in passages]
        inputs = self.reranker_tokenizer(
            [query] * len(texts), texts,
            padding=True, truncation=True, return_tensors="pt"
        )

        with torch.no_grad():
            scores = self.reranker_model(**inputs).logits.squeeze(-1)

        reranked = sorted(zip(scores.tolist(), passages), key=lambda x: -x[0])
        return [p for _, p in reranked]

    def retrieve_combined(self, original_query, reformulated_query, primary_k=5):
        enriched_query = f"Documents internes BNP : {reformulated_query}"
        passages_reformulated = self.retrieve(enriched_query, k=primary_k)
        passages_original = self.retrieve(original_query, k=primary_k)

        all_passages = passages_reformulated + passages_original
        seen = {}
        for p in all_passages:
            key = p["content_clean"]
            if key not in seen:
                seen[key] = p

        combined_passages = list(seen.values())

        if not combined_passages:
            print("[RetrieverAgent] ⚠ Aucun passage trouvé, fallback brut.")
            combined_passages = self.retrieve(original_query, k=self.fallback_k)

        reranked_passages = self.rerank(reformulated_query, combined_passages)[:primary_k]
        return reranked_passages

In [ ]:
def format_extrait(passages: list[dict]) -> str:
    result = ""
    for p in passages:
        file = p.get("file", "Document inconnu")
        page = p.get("page", "Inconnue")
        contenu = p.get("content_lite", "")
        result += (
            f"=== DOCUMENT ===\n"
            f"Nom : {file}\n"
            f"Page : {page}\n"
            f"Contenu :\n{contenu}\n\n"
        )
    return result.strip()

In [ ]:
llm_local = 'Ton API'

### Définition des agents 
explications necessaire à partir de la page 53

In [ ]:
class ReceptionAgentReformulator(Agent):
    def run(self, inputs):
        question = inputs.get("input", "").strip()
        correction_feedback = inputs.get("correction_feedback", "").strip()

        if not question:
            return {"reformulated_question": "⚠️ Aucune question reçue pour reformuler."}

        prompt = f"Reformule la question de manière claire et précise uniquement en français. Question : {question}"
        if correction_feedback:
            prompt += f"\n\nIMPORTANT : Corrige OBLIGATOIREMENT la/les erreur(s) suivante(s) :\n{correction_feedback}"

        reformulated_question = llm_local(prompt)

        return {
            "original_question": question,
            "reformulated_question": reformulated_question
        }

In [ ]:
class RechercheAgentWithRetriever(Agent):
    retriever_agent: Any = None  # Déclaré comme attribut normal, pas ClassVar !

    def run(self, inputs):
        retriever = self.retriever_agent
        original_question = inputs.get("original_question", "").strip()
        reformulated_question = inputs.get("reformulated_question", "").strip()

        with open("debug_log.txt", "a", encoding="utf-8") as f:
            f.write(f"RECHERCHE_AGENT RUN CALLED with inputs: {inputs}\n")

        passages = retriever.retrieve_combined(
            original_query=original_question,
            reformulated_query=reformulated_question,
            primary_k=7
        )

        if not passages:
            return {"context": "❌ Aucun document trouvé."}

        context = format_extrait(passages)
        return {"context": context}

In [ ]:
def split_context(context_str):
    """
    Découpe un gros contexte multi-extraits au format [file - page N]\n<contenu>
    Retourne une liste de tuples : (entête, contenu)
    """
    # Pattern : chaque extrait commence par [ ... - page ... ]
    pattern = r'(\[[^\[\]]+\])\n(.*?)(?=\n\n\[|\Z)'  # non greedy, jusqu'au prochain [ ...] ou fin
    matches = re.findall(pattern, context_str, flags=re.DOTALL)
    # Nettoie chaque extrait
    cleaned = []
    for entete, contenu in matches:
        entete = entete.strip()
        contenu = contenu.strip().replace('\n', ' ')
        if contenu:  # ignore les vides
            cleaned.append((entete, contenu))
    return cleaned

In [ ]:
class AnalyseAgentWithSummarizer(Agent):
    def run(self, inputs):
        context = inputs.get("context", "").strip()
        original_question = inputs.get("original_question", "").strip()
        reformulated_question = inputs.get("reformulated_question", "").strip()
        correction_feedback = inputs.get("correction_feedback", "").strip()

        if not context:
            print("[Analyse Agent] ⚠️ Contexte vide reçu.")
            return {
                "summary": "⚠ Aucun contenu à résumer.",
                "extraits_pertinents": []
            }

        # Découpe les extraits du contexte
        extraits = split_context(context)
        prompt_extraits = "\n\n".join(f"{entete}\n{contenu}" for entete, contenu in extraits)

        prompt = (
            "Tu es un analyste expert chargé de synthétiser un ou plusieurs documents en fonction de la question posée.\n"
            "Tu reçois une série d'extraits de documents (identifiés par entête). Pour chaque extrait, il est crucial que tu te concentres sur la **pertinence par rapport à la question**.\n"
            "Ton but est de sélectionner uniquement les extraits qui aident à répondre à la question posée. Les autres extraits doivent être ignorés.\n"
            "Commence ta réponse par une **liste brute des extraits pertinents** et indique clairement leur source (document, page, entête). Ensuite, rédige une **synthèse claire et structurée** en te basant uniquement sur les extraits retenus.\n\n"
            f"Question originale : {original_question}\n"
            f"Question reformulée : {reformulated_question}\n\n"
            "Consignes supplémentaires :\n"
            "- Si la question porte sur un aspect spécifique (par exemple, un détail d'un processus), ne garde que les extraits qui répondent directement à cet aspect.\n"
            "- Si la question demande une synthèse ou une vue d'ensemble, assure-toi de bien couvrir l'intégralité du sujet sans manquer d'éléments importants.\n"
            "- Exclue toute information qui ne répond pas directement à la question.\n"
        )

        # Ajout du feedback s'il existe
        if correction_feedback:
            prompt += f"\nIMPORTANT : Prends en compte ce retour pour améliorer ta sélection ou ta synthèse :\n{correction_feedback}\n"

        prompt += (
            f"\n=== Extraits à analyser ===\n{prompt_extraits}\n=== Fin des extraits ===\n"
            "\nFORMAT DE RÉPONSE STRICT :\n"
            "```EXTRAITS_PERTINENTS\n"
            "Colle ici la liste brute des extraits retenus (un par bloc, sans synthèse)\n"
            "```\n"
            "```SYNTHÈSE\n"
            "Donne ici la synthèse claire et structurée, basée uniquement sur ces extraits\n"
            "```"
        )



        response = llm_local(prompt)

        # Extraction auto des deux sections (si bien formatées)
        import re
        m_extr = re.search(r"```EXTRAITS_PERTINENTS\s*([\s\S]+?)```", response)
        m_syn = re.search(r"```SYNTHÈSE\s*([\s\S]+?)```", response)
        extraits_pertinents = m_extr.group(1).strip() if m_extr else ""
        synthese = m_syn.group(1).strip() if m_syn else response.strip()

        return {
            "summary": synthese,
            "extraits_pertinents": extraits_pertinents
        }

In [ ]:
class RedactionAgentWithPrompt(Agent):
    def run(self, inputs):
        original_question = inputs.get("original_question", "")
        reformulated_question = inputs.get("reformulated_question", "")
        summary = inputs.get("summary", "")
        context_brut = inputs.get("context_brut", "")
        extraits_pertinents = inputs.get("extraits_pertinents", "")  # <--- NOUVEAU
        correction_feedback = inputs.get("correction_feedback", "").strip()

        # Utilise en priorité les extraits pertinents, sinon le résumé/context_brut
        if extraits_pertinents and extraits_pertinents.strip():
            base_contexte = extraits_pertinents
            contexte_label = "Extraits pertinents à utiliser"
        elif context_brut and context_brut.strip():
            base_contexte = context_brut
            contexte_label = "Contexte brut à utiliser"
        elif summary and summary.strip():
            base_contexte = summary
            contexte_label = "Résumé synthétique à utiliser"
        else:
            return {"answer": "⚠ Contexte insuffisant pour générer une réponse fiable."}

        prompt = (
            "Tu es un expert métier. Rédige une réponse complète, structurée et strictement fondée sur les extraits pertinents suivants.\n\n"
            f"{contexte_label} :\n{base_contexte}\n\n"
            f"Résumé général pour contexte :\n{summary}\n\n"
            f"Question originale : {original_question}\n"
            f"Question reformulée : {reformulated_question}\n\n"
            "Consignes importantes :\n"
            "- Rédige une réponse structurée (paragraphes ou bullet points si besoin)\n"
            "- Chaque information doit être reliée à un extrait, cite la source (entête, page, doc)\n"
            "- Si un point n’est pas présent dans les extraits, précise-le honnêtement\n"
            "- N’invente rien, ne complète pas hors des extraits\n"
            "- Sois factuel, professionnel, précis\n"
        )
        if correction_feedback:
            prompt += f"\nIMPORTANT : Prends en compte ce retour pour améliorer ta réponse :\n{correction_feedback}\n"

        prompt += "\nDonne directement la réponse :"

        response = llm_local(prompt)
        return {"answer": response}

In [ ]:
class VerificationAgent(Agent):
    def run(self, inputs):
        task_description = inputs.get("task_description", "")
        output = inputs.get("output", "")
        original_question = inputs.get("original_question", "")
        step = inputs.get("step", "")  # Ajoute ce champ dans le manager à chaque étape

        if step == "reformulation":
            prompt = (
                f"Tâche : {task_description}\n"
                f"Question d'origine : {original_question}\n"
                f"Réponse produite : {output}\n\n"
                "Vérifie les points suivants :\n"
                "- La question est claire et sans faute\n"
                "- Elle n'est pas strictement identique à la question d'origine\n"
                "- Elle ne dénature pas le sens\n"
                "Réponds OUI si c'est correct, NON sinon (en expliquant très brièvement pourquoi si NON)."
            )
        elif step == "recherche":
            prompt = (
                f"Tâche : {task_description}\n"
                f"Question : {original_question}\n"
                f"Contexte/document trouvé : {output}\n\n"
                "Valide si le contexte/document trouvé contient au moins un élément pertinent pour répondre à la question."
                "Sinon, réponds NON (explique très brièvement pourquoi)."
            )
        elif step == "analyse":
            prompt = (
                f"Tâche : {task_description}\n"
                f"Résumé produit : {output}\n"
                f"Question de départ : {original_question}\n\n"
                "Valide si le résumé synthétise réellement les éléments du contexte, ne se contente pas de généralités, et permet de comprendre la réponse.\n"
                "Réponds OUI ou NON (en expliquant pourquoi si NON)."
            )
        elif step == "redaction":
            prompt = (
                f"Tâche : {task_description}\n"
                f"Réponse finale produite : {output}\n"
                f"Question d'origine : {original_question}\n\n"
                "Valide si la réponse est claire, bien rédigée, sourcée (si possible), ne dévie pas du contexte fourni.\n"
                "Réponds OUI ou NON (en expliquant pourquoi si NON)."
            )
        else:
            prompt = (
                f"Tâche : {task_description}\n"
                f"Entrée : {output}\n"
                "Valide uniquement si la sortie correspond à la tâche, sinon NON."
            )

        # Appel LLM
        response = llm_local(prompt)
        is_valid = "oui" in response.lower()
        return {"valid": is_valid, "feedback": response}


In [ ]:
class ConformiteAgent(Agent):
    def run(self, inputs):
        # Entrées attendues :
        # - 'documents': liste de dicts [{"filename": str, "content": str}, ...]
        # - 'procedure': texte décrivant la procédure ou exigences à vérifier
        # - 'user_instruction': consigne ou question utilisateur ("vérifie conformité", etc.)
        # - 'correction_feedback' : pour la boucle de correction éventuelle

        documents = inputs.get("documents", [])
        procedure = inputs.get("procedure", "")
        user_instruction = inputs.get("user_instruction", "")
        if not user_instruction:
            user_instruction = inputs.get("original_question", "") or inputs.get("question", "")
        correction_feedback = inputs.get("correction_feedback", "").strip()

        # Sécurité
        if not documents or not procedure:
            return {"conformite": "⚠️ Impossible de vérifier la conformité sans document(s) ET procédure à comparer."}

        # Construction du prompt
        prompt = (
            "Tu es un expert conformité réglementaire.\n\n"
            f"Procédure ou exigences à vérifier :\n{procedure}\n\n"
            "Documents soumis à l’étude :\n"
        )
        for doc in documents:
            prompt += f"\n-----\nNom : {doc.get('filename', 'document')}\nContenu :\n{doc.get('content', '')[:1200]}{'...' if len(doc.get('content', '')) > 1200 else ''}\n"

        prompt += (
            f"\nConsigne utilisateur : {user_instruction}\n"
            "\n\nÉvalue précisément la conformité de chaque document vis-à-vis de la procédure/exigences données.\n"
            "Pour chaque non-conformité détectée, cite le passage/document concerné et explique la cause."
            "\nDonne un verdict global : Conforme / Non conforme. Sois factuel et précis."
        )

        if correction_feedback:
            prompt += f"\nIMPORTANT : Prends en compte le(s) retour(s) suivant(s) pour corriger ta réponse :\n{correction_feedback}\n"

        # Appel LLM
        result = llm_local(prompt)
        return {
            "conformite": result
        }

In [ ]:
class SyntheseAgent(Agent):
    def run(self, inputs):
        """
        Entrées attendues :
            - 'documents' : liste de dicts [{"filename": str, "content": str}, ...]
            - 'user_instruction' : string libre (optionnel, par ex. "résumer", "expliquer les points-clés", etc.)
            - 'correction_feedback' : string (retour manager, optionnel)
        """
        documents = inputs.get("documents", [])
        user_instruction = inputs.get("user_instruction", "")
        if not user_instruction:
            user_instruction = inputs.get("original_question", "") or inputs.get("question", "")
        correction_feedback = inputs.get("correction_feedback", "").strip()

        if not documents:
            return {"synthese": "⚠️ Aucun document fourni à synthétiser."}

        prompt = (
            "Tu es un assistant chargé de lire et de synthétiser un ou plusieurs documents pour l'utilisateur.\n\n"
            "Consigne : Résume chaque document (ou l'ensemble si c'est cohérent), en identifiant les points-clés, les informations essentielles, les parties importantes, et toute spécificité qui pourrait intéresser un professionnel.\n"
        )
        if user_instruction:
            prompt += f"\nInstruction supplémentaire utilisateur : {user_instruction}\n"
        prompt += "\nDocuments soumis :\n"
        for doc in documents:
            prompt += f"\n-----\nNom : {doc.get('filename','document')}\nContenu :\n{doc.get('content','')[:1200]}{'...' if len(doc.get('content','')) > 1200 else ''}\n"
        if correction_feedback:
            prompt += f"\nIMPORTANT : Prends en compte le(s) retour(s) suivant(s) pour corriger/améliorer ta synthèse :\n{correction_feedback}\n"

        prompt += (
            "\nTa réponse doit comporter :\n"
            "- Un résumé global, puis un point par document (s’il y en a plusieurs)\n"
            "- Les notions, procédures, ou passages saillants\n"
            "- Si possible, identifie les thèmes/procédures abordés (ex : conformité, technique, RH, etc.)"
        )

        # Appel LLM
        result = llm_local(prompt)
        return {"synthese": result}


In [ ]:
class ExigencesAgent(Agent):
    def run(self, inputs):
        """
        Entrées attendues :
            - 'procedure_context' : str (texte du process/procédure interne récupéré)
            - 'user_instruction' : str (optionnel, consigne ou contrainte métier)
            - 'correction_feedback' : str (optionnel, feedback du vérificateur)
        """
        procedure_context = inputs.get("procedure_context", "").strip()
        user_instruction = inputs.get("user_instruction", "")
        if not user_instruction:
            user_instruction = inputs.get("original_question", "") or inputs.get("question", "")
        correction_feedback = inputs.get("correction_feedback", "").strip()

        if not procedure_context:
            return {"exigences": "⚠️ Aucun contexte de procédure fourni pour identifier les exigences."}

        prompt = (
            "Voici un extrait décrivant une procédure, une politique, une instruction interne, ou une réglementation."
            "\nAnalyse ce texte et identifie clairement toutes les exigences, obligations, mentions légales, étapes clés ou conditions nécessaires à la conformité du document ou de la procédure."
        )
        if user_instruction:
            prompt += f"\nContrainte supplémentaire à prendre en compte : {user_instruction}"
        if correction_feedback:
            prompt += f"\nIMPORTANT : Corrige selon ce feedback : {correction_feedback}"

        prompt += (
            f"\n\n=== Procédure à analyser ===\n{procedure_context[:2000]}"
            f"{'...' if len(procedure_context) > 2000 else ''}\n"
        )
        prompt += (
            "\nRéponds uniquement sous forme de liste :"
            "\n- Chaque exigence sous forme d’une phrase claire et concise"
            "\n- Numérote si possible (1., 2., ...)"
            "\n- Si aucune exigence n’est trouvée, indique-le explicitement."
        )

        result = llm_local(prompt)
        # Sécurise la sortie : ne jamais bloquer le pipeline
        if not result or not result.strip():
            result = "Aucune exigence trouvée dans la procédure."
        return {"exigences": result}


In [ ]:
class IdentificationProcedureAgent(Agent):
    def run(self, inputs):
        synthese = inputs.get("synthese", "")
        user_instruction = inputs.get("user_instruction", "")
        # Tu peux aussi passer les documents si tu veux regarder les titres, etc.

        prompt = (
            "Lis la synthèse/document ci-dessous et, en fonction du texte ET de la consigne utilisateur, "
            "identifie la procédure ou le standard interne (banque, assurance, RH, etc.) auquel il doit être comparé. "
            "Donne uniquement le nom/type de procédure à rechercher (ex : 'Ouverture de compte', 'Procédure KYC', 'Mise à jour dossier client', etc.).\n"
            f"Synthèse/document :\n{synthese}\n"
            f"Consigne utilisateur : {user_instruction}\n"
            "Réponds par une courte phrase du type : 'Procédure à rechercher : ...'"
        )
        return {"procedure_a_rechercher": llm_local(prompt)}

### instanciation denos agents

In [ ]:
# instanciation des agents CrewAI
agent_reception = ReceptionAgentReformulator(
    name="idir l'acceuil",
    role="Reformule la question uniquement en français.",
    goal="Comprendre l’intention de l’utilisateur et reformuler la question de façon claire et précise uniquement en français.",
    backstory="Expert NLP.",
    verbose=False,
    llm=llm_local
)

# 1. Crée les retriever_agents par domaine
retriever_agents = {}
for domain in indices.keys():
    retriever_agents[domain] = RetrieverAgent(
        model=model,
        index=indices[domain],
        nlp=nlp,
        indexed_chunks=chunks[domain],
        reranker_model=reranker_model,
        reranker_tokenizer=reranker_tokenizer
    )

# 2. Crée les agents de recherche par domaine
recherche_agents = {}
for domain in retriever_agents.keys():
    agent = RechercheAgentWithRetriever(
        retriever_agent=retriever_agents[domain],
        name=f"chercheur_{domain}",
        role=f"Recherche des documents pour le domaine {domain}",
        goal="Trouver les documents les plus pertinents pour le domaine " + domain,
        backstory="Expert documentaire.",
        verbose=False,
        llm=llm_local
    )
    recherche_agents[domain] = agent

agent_analyse = AnalyseAgentWithSummarizer(
    name="yasmina l'analyste",
    role="Filtre, sélectionne et synthétise les extraits pertinents issus des documents, uniquement en français.",
    goal=(
        "Parmi tous les extraits issus des documents, sélectionner ceux qui sont directement pertinents pour la question, "
        "et produire une synthèse claire, structurée, en citant les extraits utiles. "
        "La synthèse doit être exhaustive, fidèle et uniquement basée sur les extraits sélectionnés."
    ),
    backstory=(
        "Analyste documentaire senior, spécialisé dans la synthèse experte de corpus volumineux. "
        "Ton expertise est d’identifier l’information critique dans des ensembles de documents et de la restituer "
        "de manière structurée et fiable pour des utilisateurs exigeants (banque, juridique, data science)."
    ),
    verbose=False,
    llm=llm_local
)

agent_redaction = RedactionAgentWithPrompt(
    name="nihad la rédactrice",
    role="Rédige une réponse claire et sourcée uniquement en français.",
    goal="Produire une réponse claire et sourcée basée sur le contexte fourni uniquement en français.",
    backstory="Assistant IA spécialisé.",
    verbose=False,
    llm=llm_local
)

agent_verificateur = VerificationAgent(
    name="Kenza la vérificatrice",
    role="Vérifie si l’agent a bien accompli sa tâche.",
    goal="Assurer la conformité de chaque tâche agent avec la consigne donnée.",
    backstory="Auditeur IA expert.",
    verbose=False,
    llm=llm_local
)

exigences_agent = ExigencesAgent(
    name="Agent Exigences",
    role="Identifie et extrait les exigences (règles, obligations, étapes…) à partir d’une procédure interne ou d’un standard.",
    goal="Analyser le texte d’une procédure ou d’une politique interne pour extraire la liste précise des exigences à respecter.",
    backstory="Expert conformité et analyse réglementaire.",
    verbose=False,    # Debug désactivé
    llm=llm_local     # Ton instance LLM (locale ou API)
)

synthese_agent = SyntheseAgent(
    name="Agent Synthèse",
    role="Analyse et synthétise des documents importés.",
    goal="Lire un ou plusieurs documents fournis par l’utilisateur et produire un résumé structuré, des points-clés et, si besoin, des extraits.",
    backstory="Assistant IA expérimenté en lecture rapide et extraction d’informations.",
    verbose=False,   # On désactive le debug/print
    llm=llm_local    # Ton instance/handle de LLM locale ou API
)

conformite_agent = ConformiteAgent(
    name="Agent Conformité",
    role="Vérifie la conformité documentaire/procédurale.",
    goal="Analyser les documents importés et vérifier leur conformité avec les procédures internes.",
    backstory="Expert conformité réglementaire.",
    verbose=False,  # Désactive le mode bavard en prod
    llm=llm_local
)

agent_identification_procedure = IdentificationProcedureAgent(
    name="Detecteur de Procédure",
    role="Détecte la procédure ou le standard à vérifier à partir des documents importés.",
    goal="Analyser le document et la consigne utilisateur pour déterminer la procédure de référence.",
    backstory="Assistant IA spécialisé dans l’orientation des requêtes conformité.",
    verbose=False,
    llm=llm_local
)



In [ ]:
class MPCAgent(Agent):
    def run(self, inputs):
        # On préfère "original_question" pour garder la logique globale
        question = inputs.get("original_question", "") or inputs.get("question", "")
        documents = inputs.get("documents", [])
        domains = inputs.get("domains", [])
        doc_names = [doc.get("filename", "doc") for doc in documents][:3]
        doc_info = ", ".join(doc_names) if doc_names else "aucun"

        prompt = (
            "Tu es l'agent chef d’orchestre d’un assistant IA multi-agent.\n"
            "À chaque question, tu dois DÉCIDER la pipeline la plus adaptée, en expliquant ton choix.\n"
            "Tu dois TOUJOURS répondre en JSON STRICT.\n"
            "Tu as 4 pipelines possibles :\n"
            "- 'rag' : Recherche interne uniquement (pas de documents importés)\n"
            "- 'explication' : Explication/synthèse de documents importés (l'utilisateur veut qu’on explique ou analyse ses fichiers)\n"
            "- 'conformite' : Vérification de conformité des docs importés vs une procédure\n"
            "- 'mixte' : Croisement des infos docs importés + base interne (si la question le demande)\n"
            "\nRends TOUJOURS une réponse JSON au format :\n"
            "{\n"
            "  \"pipeline\": \"rag|explication|conformite|mixte\",\n"
            "  \"domaine\": \"...\",\n"
            "  \"need_reformulation\": true|false,\n"
            "  \"steps_plan\": [\"étape1\", \"étape2\", ...],\n"
            "  \"feedback\": \"explique pourquoi ce choix, mentionne les indices dans la question ou la présence/absence de documents\"\n"
            "}\n"
            "Exemples :\n"
            "- question : 'explique ces documents', docs OUI → pipeline='explication'\n"
            "- question : 'vérifie la conformité', docs OUI → pipeline='conformite'\n"
            "- question : 'donne la procédure X', docs NON → pipeline='rag'\n"
            "- question : 'croise ce doc avec la base', docs OUI → pipeline='mixte'\n"
            "\nINPUTS :\n"
            f"- Question : {question}\n"
            f"- Docs importés ({len(documents)}) : {doc_info}\n"
            f"- Domaines disponibles : {', '.join(domains)}\n"
        )

        result_str = llm_local(prompt)
        import json, re
        # Essaye de parser proprement le JSON
        try:
            result_json_str = re.search(r"\{.*\}", result_str, re.DOTALL).group(0)
            result = json.loads(result_json_str)
        except Exception:
            # Fallback (pour éviter le crash)
            result = {
                "pipeline": "rag",
                "domaine": domains[0] if domains else "général",
                "need_reformulation": True,
                "steps_plan": [],
                "feedback": "Fallback: parsing échoué, prompt non respecté."
            }
        # Sécurise les champs
        result["pipeline"] = result.get("pipeline", "rag")
        result["domaine"] = result.get("domaine", domains[0] if domains else "général")
        result["need_reformulation"] = (
            str(result.get("need_reformulation", "True")).strip().lower() in ["true", "oui", "yes"]
        )
        result["steps_plan"] = result.get("steps_plan", [])
        result["feedback"] = result.get("feedback", "RAS")
        return result

In [ ]:
agent_mpc = MPCAgent(
    name="Samy le MPC",
    role="Oriente la question, choisit le domaine pertinent et la stratégie.",
    goal="Analyser la question pour choisir le bon domaine et le flux optimal.",
    backstory="Orchestrateur intelligent, expert du contexte métier.",
    verbose=False,
    llm=llm_local
)

In [ ]:
def extract_reformulation(text):
    m = re.search(r'["“«]([^"”»]+)["”»]', text)
    if m:
        return m.group(1).strip()
    if ":" in text:
        last = text.split(":")[-1].strip()
        return last if last else text.strip()
    return text.strip()

def extract_context(text):
    return text.strip()

def extract_summary(text):
    return text.strip()

def extract_answer(text):
    return text.strip()

def is_acceptable_reformulation(original, reformulated):
    # Considère acceptable si ce n'est pas strictement identique et pas vide
    if not reformulated or not original:
        return False
    return reformulated.strip().lower() != original.strip().lower()

### mis en plage de l'orchestrateur 

In [ ]:
class AgentManager:
    def __init__(self, agents, agent_verif, agent_mpc, recherche_agents, domains, max_retries=2):

        self.agents = agents
        self.agent_verif = agent_verif
        self.agent_mpc = agent_mpc
        self.recherche_agents = recherche_agents
        self.domains = domains
        self.max_retries = max_retries
        self.log_steps = []

    def log(self, msg):
        # Tu peux simplement passer si tu ne veux pas de logs
        pass

    def safe_agent_run(self, agent, input_dict, task_description, user_question, extractor, step):
        feedback = ""
        agent_output = None
        verif = None
        for attempt in range(self.max_retries + 1):
            if feedback:
                input_dict['correction_feedback'] = feedback
            agent_output = agent.run(input_dict)
            if isinstance(agent_output, dict):
                value = next((v for v in agent_output.values() if isinstance(v, str) and v.strip()), "")
            else:
                value = str(agent_output)
            extracted = extractor(value)
            verif_input = {
                "task_description": task_description + (f" (Corrige selon feedback : {feedback})" if feedback else ""),
                "output": extracted,
                "original_question": user_question,
                "step": step
            }
            verif = self.agent_verif.run(verif_input)
            valid = verif.get('valid', False)
            self.log(
                f"Essai {attempt+1} / {self.max_retries+1} pour {task_description} : valid={valid}\nFeedback: {verif.get('feedback','')}\nRéponse candidate: {repr(extracted)}"
            )
            if valid:
                return extracted, verif, attempt + 1
            feedback = verif.get('feedback', "")
        return extracted, verif, self.max_retries + 1

    def run_pipeline(self, user_question, documents=None, user_instruction=None):
        """
        user_question : str
        documents : list de dicts [{"filename":..., "content":...}], optionnel
        """
        # --- Initialisation du résultat global
        result = {
            "mpc_decision": {},
            "pipeline": "",
            "domaine": "",
            "need_reformulation": False,
            "steps_plan": [],
            "feedback_mpc": "",
            "reformulation": "",
            "context": "",
            "summary": "",
            "answer": "",
            "exigences": "",
            "conformite": "",
            "verification": [],
            "success": False,
            "attempts": 0,
            "reason": "",
        }

        # ==== 1. Appel MPC ====
        mpc_inputs = {
            "original_question": user_question,
            "documents": documents if documents else [],
            "domains": self.domains
        }
        mpc_result = self.agent_mpc.run(mpc_inputs) or {}

        # Extraction (avec fallback par défaut)
        pipeline = mpc_result.get("pipeline", "rag")
        domaine = mpc_result.get("domaine", self.domains[0] if self.domains else "général")
        need_reformulation = mpc_result.get("need_reformulation", True)
        steps_plan = mpc_result.get("steps_plan", [])
        feedback_mpc = mpc_result.get("feedback", "")

        # Stocke tout dans le résultat global
        result["mpc_decision"] = mpc_result
        result["pipeline"] = pipeline
        result["domaine"] = domaine
        result["need_reformulation"] = need_reformulation
        result["steps_plan"] = steps_plan
        result["feedback_mpc"] = feedback_mpc

        reformulation = user_question
        n_attempts_reform = 1
        verif_reform = {"valid": True, "feedback": ""}
        if need_reformulation:
            # On prépare l'input pour l'agent reception/reformulateur
            reform_input = {
            "input": user_question
            }
            # Appel de l'agent de reformulation
            reformulation_result = self.agents["reception"].run(reform_input)
            # Essaye de récupérer la question reformulée selon ton agent
            if isinstance(reformulation_result, dict) and "reformulated_question" in reformulation_result:
                candidate = reformulation_result["reformulated_question"]
                # Optionnel : si agent vérificateur, le passer ici aussi pour valider la reformulation
                verif_input = {
                    "task_description": "Vérifie la qualité de la reformulation.",
                    "output": candidate,
                    "original_question": user_question,
                    "step": "reformulation"
                }
                verif_reform = self.agent_verif.run(verif_input)
                # Si la reformulation est vide ou trop proche de l'original, fallback sur l'original
                if not verif_reform.get("valid", False):
                    from difflib import SequenceMatcher
                    similarity = SequenceMatcher(None, candidate, user_question).ratio()
                    if similarity > 0.95 or not candidate.strip():
                        reformulation = user_question  # on garde l'original
                        verif_reform["feedback"] = verif_reform.get("feedback", "") + " (Reformulation ignorée, trop similaire ou vide)"
                    else:
                        reformulation = candidate
                else:
                    reformulation = candidate
            else:
                reformulation = user_question  # fallback

        # Ajoute au résultat global
        result["reformulation"] = reformulation
        result["verification"].append({
            "step": "reformulation",
            "feedback": verif_reform.get("feedback", ""),
            "valid": verif_reform.get("valid", True),
            "attempts": n_attempts_reform
        })

            # === 3. Sélection pipeline selon la décision de l’agent MPC ===
        pipeline_type = mpc_result.get("pipeline", "rag").lower()  # par défaut "rag"
        domain = mpc_result.get("domaine", self.domains[0])
        agent_recherche = self.recherche_agents.get(domain) or next(iter(self.recherche_agents.values()))
        # (On peut gérer d'autres variables issues du MPC si besoin)

        if pipeline_type == "rag":
            # -- 2. Recherche documentaire (pipeline RAG classique)
            context, verif_search, n_attempts_search = self.safe_agent_run(
                agent_recherche,
                {
                    "original_question": user_question,
                    "reformulated_question": reformulation
                },
                f"Trouve les documents pertinents pour répondre à la question dans le domaine {domain}.",
                user_question,
                extract_context,
                "recherche"
            )
            result["context"] = context
            result["verification"].append({
                "step": "recherche",
                "feedback": verif_search.get("feedback", ""),
                "valid": verif_search.get("valid", False),
                "attempts": n_attempts_search
            })
            # Pas de blocage si pas de contexte pertinent
            if not verif_search.get("valid", False) or not context.strip():
                result["reason"] = f"Aucun contexte/document pertinent trouvé ou blocage vérificateur après {n_attempts_search} essais : {verif_search.get('feedback', '')}"

            # -- 3. Analyse/synthèse des extraits
            summary, verif_analyse, n_attempts_analyse = self.safe_agent_run(
                self.agents["analyse"],
                {
                    "context": context,
                    "original_question": user_question,
                    "reformulated_question": reformulation
                },
                "Synthétise et résume les éléments pertinents des documents trouvés pour répondre à la question.",
                user_question,
                extract_summary,
                "analyse"
            )
            result["summary"] = summary
            result["verification"].append({
                "step": "analyse",
                "feedback": verif_analyse.get("feedback", ""),
                "valid": verif_analyse.get("valid", False),
                "attempts": n_attempts_analyse
            })
            # Toujours continuer même si le résumé n’est pas parfait

            # -- 4. Rédaction finale
            answer, verif_redac, n_attempts_redac = self.safe_agent_run(
                self.agents["redaction"],
                {
                    "original_question": user_question,
                    "reformulated_question": reformulation,
                    "summary": summary,
                    "context_brut": context
                },
                "Rédige une réponse claire, précise et sourcée, uniquement en français, sur la base du résumé fourni.",
                user_question,
                extract_answer,
                "redaction"
            )
            result["answer"] = answer
            result["verification"].append({
                "step": "redaction",
                "feedback": verif_redac.get("feedback", ""),
                "valid": verif_redac.get("valid", False),
                "attempts": n_attempts_redac
            })
            # Même en cas d'échec, la pipeline va jusqu’au bout, pas de blocage

            # -- Statut global et message final
            result["success"] = True
            result["attempts"] = n_attempts_reform + n_attempts_search + n_attempts_analyse + n_attempts_redac
            result["reason"] = result.get("reason", "Pipeline RAG exécutée (avec ou sans blocage).")

        elif pipeline_type == "conformite":
            # 1. Lecture/Synthèse des documents importés
            synthese, verif_synth, n_attempts_synth = self.safe_agent_run(
                self.agents["synthese"],  # <-- Ajoute-le bien à self.agents lors de l'instanciation du manager
                {
                    "documents": documents,  # tu dois faire passer la liste de dicts ici
                    "user_instruction": user_question  # l'instruction posée dans la barre
                },
                "Fais une synthèse structurée des documents importés.",
                user_question,
                lambda x: x,  # Pas d'extracteur spécifique, résultat déjà dict avec "synthese"
                "synthese"
            )
            result["synthese"] = synthese
            result["verification"].append({
                "step": "synthese",
                "feedback": verif_synth.get("feedback", ""),
                "valid": verif_synth.get("valid", False),
                "attempts": n_attempts_synth
            })

            # 2. Recherche/extraction de la procédure ou des exigences
            # Ici, tu peux utiliser un agent Recherche sur un domaine précis, ou un agent Exigences dédié
            procedure_context = ""  # à remplir selon comment tu trouves la procédure (ex via agent recherche ou user input)
            exigences = ""
            if "agent_exigences" in self.agents:
                exigences, verif_exig, n_attempts_exig = self.safe_agent_run(
                    self.agents["exigences"],
                    {
                        "procedure_context": procedure_context,  # Met ici le texte à analyser !
                        "user_instruction": user_question
                    },
                    "Identifie toutes les exigences à respecter pour la procédure.",
                    user_question,
                    lambda x: x,
                    "exigences"
                )
                result["exigences"] = exigences
                result["verification"].append({
                    "step": "exigences",
                    "feedback": verif_exig.get("feedback", ""),
                    "valid": verif_exig.get("valid", False),
                    "attempts": n_attempts_exig
                })
            else:
                # Cas où pas d'agent exigences, tu peux chercher via agent_recherche sur "procédures"
                pass

            # 3. Vérification de conformité proprement dite
            conformite, verif_conf, n_attempts_conf = self.safe_agent_run(
                self.agents["conformite"],
                {
                    "documents": documents,
                    "procedure": exigences if exigences else procedure_context,
                    "user_instruction": user_question
                },
                "Vérifie la conformité des documents importés vis-à-vis des exigences/processus internes.",
                user_question,
                lambda x: x,
                "conformite"
            )
            result["conformite"] = conformite
            result["verification"].append({
                "step": "conformite",
                "feedback": verif_conf.get("feedback", ""),
                "valid": verif_conf.get("valid", False),
                "attempts": n_attempts_conf
            })

            # 4. Rédaction finale (sur le retour conformité, ou synthèse…)
            answer, verif_redac, n_attempts_redac = self.safe_agent_run(
                self.agents["redaction"],
                {
                    "original_question": user_question,
                    "summary": synthese,
                    "context_brut": str(conformite),
                    "extraits_pertinents": "",  # Optionnel si besoin
                },
                "Rédige un rapport clair et factuel sur la conformité des documents.",
                user_question,
                extract_answer,
                "redaction"
            )
            result["answer"] = answer
            result["verification"].append({
                "step": "redaction",
                "feedback": verif_redac.get("feedback", ""),
                "valid": verif_redac.get("valid", False),
                "attempts": n_attempts_redac
            })

            result["success"] = True
            result["attempts"] = n_attempts_synth + (n_attempts_exig if "agent_exigences" in self.agents else 0) + n_attempts_conf + n_attempts_redac
            result["reason"] = result.get("reason", "Pipeline conformité exécutée.")
        
        elif pipeline_type == "explication":
            # 1. Synthèse ou lecture des documents (si présents)
            if documents:
                synthese, verif_synth, n_attempts_synth = self.safe_agent_run(
                    self.agents["synthese"],
                    {
                        "documents": documents,
                        "user_instruction": user_question  # tu passes l’instruction telle quelle
                    },
                    "Fais une synthèse/lecture des documents importés pour fournir du contexte à l'explication.",
                    user_question,
                    lambda x: x,
                    "synthese"
                )
                base_context = synthese
                result["synthese"] = synthese
                result["verification"].append({
                    "step": "synthese",
                    "feedback": verif_synth.get("feedback", ""),
                    "valid": verif_synth.get("valid", False),
                    "attempts": n_attempts_synth
                })
            else:
                base_context = ""

            # 2. Agent "explication" (tu peux soit dédier un agent, soit utiliser l'agent analyse avec un prompt spécifique)
            # Si tu veux un agent dédié :
            if "explication" in self.agents:
                explication, verif_expl, n_attempts_expl = self.safe_agent_run(
                    self.agents["explication"],
                    {
                        "context": base_context,
                        "user_instruction": user_question
                    },
                    "Fournis une explication détaillée sur la notion ou la question posée.",
                    user_question,
                    lambda x: x,
                    "explication"
                )
                result["explication"] = explication
                result["verification"].append({
                    "step": "explication",
                    "feedback": verif_expl.get("feedback", ""),
                    "valid": verif_expl.get("valid", False),
                    "attempts": n_attempts_expl
                })
            else:
                # Sinon, fallback sur agent analyse générique
                explication, verif_expl, n_attempts_expl = self.safe_agent_run(
                    self.agents["analyse"],
                    {
                        "context": base_context,
                        "original_question": user_question,
                        "reformulated_question": user_question  # tu peux passer l’original si pas reformulée
                    },
                    "Explique la notion demandée, en s'appuyant sur les documents importés si présents.",
                    user_question,
                    lambda x: x,
                    "explication"
                )
                result["explication"] = explication
                result["verification"].append({
                    "step": "explication",
                    "feedback": verif_expl.get("feedback", ""),
                    "valid": verif_expl.get("valid", False),
                    "attempts": n_attempts_expl
                })

            # 3. Rédaction finale
            answer, verif_redac, n_attempts_redac = self.safe_agent_run(
                self.agents["redaction"],
                {
                    "original_question": user_question,
                    "summary": base_context,
                    "context_brut": result["explication"],
                    "extraits_pertinents": "",  # ou tu peux y mettre un extrait clé, selon ton output agent explication
                },
                "Rédige une explication claire, précise et pédagogique basée sur l'analyse précédente.",
                user_question,
                extract_answer,
                "redaction"
            )
            result["answer"] = answer
            result["verification"].append({
                "step": "redaction",
                "feedback": verif_redac.get("feedback", ""),
                "valid": verif_redac.get("valid", False),
                "attempts": n_attempts_redac
            })

            result["success"] = True
            result["attempts"] = (n_attempts_synth if documents else 0) + n_attempts_expl + n_attempts_redac
            result["reason"] = result.get("reason", "Pipeline explication exécutée.")

        elif pipeline_type == "mixte":
            # 1. Synthèse/lecture des documents importés
            if documents:
                synthese, verif_synth, n_attempts_synth = self.safe_agent_run(
                    self.agents["synthese"],
                    {
                        "documents": documents,
                        "user_instruction": user_question
                    },
                    "Lis et synthétise le(s) document(s) importé(s) pour préparer la réponse.",
                    user_question,
                    lambda x: x,
                    "synthese"
                )
                base_doc_context = synthese
                result["synthese"] = synthese
                result["verification"].append({
                    "step": "synthese",
                    "feedback": verif_synth.get("feedback", ""),
                    "valid": verif_synth.get("valid", False),
                    "attempts": n_attempts_synth
                })
            else:
                base_doc_context = ""

            # 2. Recherche documentaire RAG (base interne, domaine choisi par MPC)
            context, verif_search, n_attempts_search = self.safe_agent_run(
                agent_recherche,
                {
                    "original_question": user_question,
                    "reformulated_question": reformulation
                },
                f"Trouve les documents internes pertinents pour répondre à la question (domaine : {domain}).",
                user_question,
                extract_context,
                "recherche"
            )
            result["context"] = context
            result["verification"].append({
                "step": "recherche",
                "feedback": verif_search.get("feedback", ""),
                "valid": verif_search.get("valid", False),
                "attempts": n_attempts_search
            })

            # 3. Analyse croisée (optionnel mais conseillé)
            # On peut fusionner les contextes (synthese docs + context rag) pour l’analyse finale
            analyse, verif_analyse, n_attempts_analyse = self.safe_agent_run(
                self.agents["analyse"],
                {
                    "context": (base_doc_context or "") + "\n\n" + (context or ""),
                    "original_question": user_question,
                    "reformulated_question": reformulation
                },
                "Synthétise les informations clés issues à la fois des documents importés et de la base documentaire interne.",
                user_question,
                extract_summary,
                "analyse"
            )
            result["summary"] = analyse
            result["verification"].append({
                "step": "analyse",
                "feedback": verif_analyse.get("feedback", ""),
                "valid": verif_analyse.get("valid", False),
                "attempts": n_attempts_analyse
            })

            # 4. Rédaction finale (réponse claire, enrichie, croisée)
            answer, verif_redac, n_attempts_redac = self.safe_agent_run(
                self.agents["redaction"],
                {
                    "original_question": user_question,
                    "reformulated_question": reformulation,
                    "summary": analyse,
                    "context_brut": (base_doc_context or "") + "\n\n" + (context or "")
                },
                "Rédige une réponse synthétique et enrichie, croisant l’info des documents importés et celle trouvée dans la base documentaire interne.",
                user_question,
                extract_answer,
                "redaction"
            )
            result["answer"] = answer
            result["verification"].append({
                "step": "redaction",
                "feedback": verif_redac.get("feedback", ""),
                "valid": verif_redac.get("valid", False),
                "attempts": n_attempts_redac
            })

            result["success"] = True
            result["attempts"] = (
                (n_attempts_synth if documents else 0)
                + n_attempts_search
                + n_attempts_analyse
                + n_attempts_redac
            )
            result["reason"] = result.get("reason", "Pipeline mixte exécutée.")
        
        return result


In [ ]:
agents = {
    "reception": agent_reception,
    "analyse": agent_analyse,
    "redaction": agent_redaction,
    "conformite": conformite_agent,  
    "synthese": synthese_agent,      
    "exigences": exigences_agent,
    "identification_procedure": agent_identification_procedure   
}

In [ ]:
manager = AgentManager(
    agents=agents,
    agent_verif=agent_verificateur,
    agent_mpc=agent_mpc,
    recherche_agents=recherche_agents,  
    domains=domains
)

#### exemple d'utilisation

In [ ]:
domains = detect_domains(parent_folder)
question_utilisateur = "explique moi ces document j'ai du mal avec ce domaine"
documents = [
    {
        "filename": "presentation_RAG.pdf",
        "content": (
            "Le système RAG (Retrieval-Augmented Generation) combine les capacités d'un modèle génératif avec la recherche documentaire. "
            "Il permet d'enrichir les réponses d'un assistant IA en allant puiser dans une base de connaissances structurée. "
            "Dans une architecture RAG, l'agent de recherche récupère les passages pertinents, l'agent génératif synthétise la réponse finale, "
            "et le processus s'appuie sur des embeddings pour indexer les documents."
        )
    },
    {
        "filename": "architecture_agents_IA.txt",
        "content": (
            "Un agent IA est une entité logicielle autonome capable de percevoir son environnement, raisonner, et agir pour atteindre un objectif. "
            "Dans le contexte du projet, plusieurs agents spécialisés (Recherche, Analyse, Conformité) coopèrent pour traiter les requêtes complexes. "
            "Chaque agent utilise des modèles d'intelligence artificielle pour comprendre, extraire, et reformuler l'information."
        )
    },
    {
        "filename": "note_explicative_IA.docx",
        "content": (
            "L'intelligence artificielle désigne l'ensemble des méthodes permettant à une machine d'apprendre, de raisonner, et d'agir. "
            "Les assistants basés sur l'IA utilisent le NLP, l'apprentissage automatique, et la recherche de documents (RAG) pour fournir des réponses personnalisées. "
            "L'utilisation d'agents multiples améliore la modularité, l'explicabilité, et la performance du système."
        )
    }
]


result = manager.run_pipeline(
    question_utilisateur,
    documents=documents,
)

result["answer"]